In [109]:
# WARNING! Be sure to change the line that reads the .csv file, below, should be line 47
# If it doesn't work, you'll need to make the path point to your copy of that file (which should be in this folder)

# We need to import these modules to get started
import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
from allensdk.core.brain_observatory_cache import BrainObservatoryCache

# The dynamic brain database is on an external hard drive.
#drive_path = '/Volumes/Brain2016'
drive_path= 'd:'
manifest_path = os.path.join(drive_path, 'BrainObservatory/manifest.json')
boc = BrainObservatoryCache(manifest_file=manifest_path)
expt_containers = boc.get_experiment_containers()

# Things I will want eventually:
# 1. A dataframe with columns for...
#	a. all the identifiers from the brain observatory
#	b. the preferred directions from the grating and place field stimuli
# 2. A numpy array which is N_neurons x N_timesamples, where entry i,j is 1 or 0 based on whether
#	 the neuron was "firing" at that point (how we decide that we'll get to later

# Data frames first:
# expt_containers_df = pd.DataFrame(expt_containers)

# The stimuli I'm interested in are basically everything except the natural movie and natural scenes
# If we want other stimuli, add them here.
non_movie_stimuli = ['drifting_gratings', 'locally_sparse_noise', 'spontaneous', 'static_gratings']

all_expts_df = pd.DataFrame(boc.get_ophys_experiments(stimuli=non_movie_stimuli))
# this has headers:
# age_days	cre_line	experiment_container_id	id	imaging_depth	session_type	targeted_structure

# seems like I can use get_cell_speciments to get everything I'm after
specimens_df = pd.DataFrame(boc.get_cell_specimens(experiment_container_ids=all_expts_df.experiment_container_id.values))
# this has headers:
# area	cell_specimen_id	dsi_dg	experiment_container_id	imaging_depth	osi_dg	osi_sg	p_dg	p_ns	p_sg
# pref_dir_dg	pref_image_ns	pref_ori_sg	pref_phase_sg	pref_sf_sg	pref_tf_dg	time_to_peak_ns	time_to_peak_sg
# tld1_id	tld1_name	tld2_id	tld2_name	tlr1_id	tlr1_name

# There's also a handy bit of data from Saskia, in the form of a measurement called S. See
# Decoding Visual Inputs From Multiple Neurons in the Human Temporal Lobe, J. Neurophys 2007, by Quiroga et al


selectivity_S_df = pd.read_csv(os.path.join(drive_path, '/BrainObservatory/image_selectivity_dataframe.csv'), index_col=0)
selectivity_S_df = selectivity_S_df[['cell_specimen_id', 'selectivity_ns']]

specimens_with_selectivity_S = specimens_df.merge(selectivity_S_df,how='outer', on='cell_specimen_id')

# This is all cells in VISp that have a value for the specified parameters (i.e not NaN)
# Discards rows NaN in the columns specified below.
discard_nan = [
	'selectivity_ns',
	'osi_sg',
	'osi_dg',
	'time_to_peak_ns',
	'time_to_peak_sg',
]
VISp_cells_with_numbers = specimens_with_selectivity_S[specimens_with_selectivity_S.area == 'VISp']
for col_name in discard_nan:
	VISp_cells_with_numbers = VISp_cells_with_numbers[np.isnan(VISp_cells_with_numbers[col_name])==False]

In [110]:
VISp_cells_with_numbers.head(2)

,area,cell_specimen_id,dsi_dg,experiment_container_id,imaging_depth,osi_dg,osi_sg,p_dg,p_ns,p_sg,...,pref_tf_dg,time_to_peak_ns,time_to_peak_sg,tld1_id,tld1_name,tld2_id,tld2_name,tlr1_id,tlr1_name,selectivity_ns
381,VISp,517397361,0.697331,511498742,350,1.388875,1.210908,0.000002,3.065294e-18,0.111141,...,1.0,0.33250,0.266,177837516,Scnn1a-Tg3-Cre,177837320,Camk2a-tTA,265943423,Ai93(TITL-GCaMP6f),0.586288
382,VISp,517397365,3.640992,511498742,350,0.907149,1.994455,0.002307,2.121158e-13,0.007671,...,1.0,0.29925,0.266,177837516,Scnn1a-Tg3-Cre,177837320,Camk2a-tTA,265943423,Ai93(TITL-GCaMP6f),0.513627


In [111]:
print(VISp_cells_with_numbers.keys())

Index([                   u'area',        u'cell_specimen_id',
                        u'dsi_dg', u'experiment_container_id',
                 u'imaging_depth',                  u'osi_dg',
                        u'osi_sg',                    u'p_dg',
                          u'p_ns',                    u'p_sg',
                   u'pref_dir_dg',           u'pref_image_ns',
                   u'pref_ori_sg',           u'pref_phase_sg',
                    u'pref_sf_sg',              u'pref_tf_dg',
               u'time_to_peak_ns',         u'time_to_peak_sg',
                       u'tld1_id',               u'tld1_name',
                       u'tld2_id',               u'tld2_name',
                       u'tlr1_id',               u'tlr1_name',
                u'selectivity_ns'],
      dtype='object')


In [112]:
print(VISp_cells_with_numbers["pref_image_ns"].unique())
len(VISp_cells_with_numbers["pref_image_ns"].unique())

[  17.   22.   99.   38.  111.   95.    2.   97.  106.   41.  117.   66.
  101.   49.  110.   24.   13.   50.   33.  114.   44.   92.   63.   32.
   23.   71.  105.    3.   39.   45.   29.   16.   91.   78.   69.  112.
   88.   64.   72.   56.   26.    7.   67.   89.  116.   12.  103.   83.
  115.   75.    5.   68.   48.   96.   35.   86.    6.   27.   15.  108.
   40.   51.  109.   93.   58.   76.   42.    1.   46.   36.   20.   98.
  113.   18.   52.   85.    0.   84.   80.   94.   34.   43.   21.   55.
   60.   54.  104.   90.   14.   65.   70.   82.   79.   87.   53.    9.
   25.   28.    8.   11.  100.   81.   31.   19.  102.   37.   57.   61.
   73.   77.   59.   74.   30.   62.   10.  107.    4.]


117

In [135]:
# very long, uncomment only if you wish to see number of cells that prefer EACH image

#print("# of total cells according to their preferred image: ")
#for image in sorted(VISp_cells_with_numbers["pref_image_ns"].unique()):
#    print ("prefer image #"+str(int(image))+":  "+str(len(VISp_cells_with_numbers
#                                         [VISp_cells_with_numbers["pref_image_ns"]==image]))+" cells")

In [154]:
print("cell totals per preferred image group:") 
      
image_ns_prefs = [len(VISp_cells_with_numbers[VISp_cells_with_numbers["pref_image_ns"]==image]) 
 for image in sorted(VISp_cells_with_numbers["pref_image_ns"].unique())]
image_ns_prefs.sort()
print(image_ns_prefs) #list of cells
print('')

print("total cells in all groups:")
total_ns_pref = sum(image_ns_prefs)
print(total_ns_pref)
print('')

print("largest group of cells:")
max_ns_prefs = np.max(image_ns_prefs) #max number cells with preferred static image
print(max_ns_prefs)
print('')

print("percent of total cells:")
per_ns_pref = float(max_ns_prefs)/float(total_ns_pref)
print (str(per_ns_pref)+ " = roughly "+str(round(per_ns_pref*100))+"%")
print("")

print("most preferred image(s):")
for image in sorted(VISp_cells_with_numbers["pref_image_ns"].unique()):
    if(((len(VISp_cells_with_numbers[VISp_cells_with_numbers["pref_image_ns"]==image])))==max_ns_prefs):
        print(int(image))

cell totals per preferred image group:
[2, 2, 3, 4, 4, 4, 5, 6, 7, 7, 8, 8, 8, 8, 9, 10, 10, 11, 11, 12, 12, 12, 12, 13, 14, 14, 14, 15, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 18, 20, 20, 20, 20, 21, 21, 21, 21, 22, 23, 23, 23, 24, 24, 24, 24, 25, 25, 26, 26, 26, 27, 27, 27, 28, 28, 28, 29, 30, 30, 30, 30, 30, 32, 32, 32, 33, 34, 34, 35, 35, 36, 37, 37, 37, 37, 37, 38, 38, 38, 39, 39, 40, 41, 42, 48, 49, 49, 49, 51, 51, 52, 52, 56, 62, 63, 63, 64, 64, 65, 65, 66, 67, 72, 74, 79, 83, 83]

total cells in all groups:
3483

largest group of cells:
83

percent of total cells:
0.023830031582 = roughly 2.0%

Most preferred image(s):
86
111


In [120]:
print("List of unique orientations for static gratings: ")
print(VISp_cells_with_numbers["pref_ori_sg"].unique())
print("")
print("# of unique orientations for static gratings: ")
print(len(VISp_cells_with_numbers["pref_ori_sg"].unique()))

List of unique orientations for static gratings: 
[  60.    0.   30.  120.  150.   90.]

# of unique orientations for static gratings: 
6


In [121]:
print("# of total cells according to their preferred orientation: ")
for orientation in sorted(VISp_cells_with_numbers["pref_ori_sg"].unique()):
    print ("prefer "+str(int(orientation))+" degrees: "+str(len(VISp_cells_with_numbers
                                         [VISp_cells_with_numbers["pref_ori_sg"]==orientation]))+" cells")

# of total cells according to their preferred orientation: 
prefer 0 degrees: 580 cells
prefer 30 degrees: 600 cells
prefer 60 degrees: 551 cells
prefer 90 degrees: 690 cells
prefer 120 degrees: 568 cells
prefer 150 degrees: 494 cells


In [156]:
print("cell totals per preferred orientation group:")

ori_sg_prefs = [len(VISp_cells_with_numbers[VISp_cells_with_numbers["pref_ori_sg"]==orientation]) 
 for orientation in sorted(VISp_cells_with_numbers["pref_ori_sg"].unique())]
ori_sg_prefs.sort()
print(ori_sg_prefs) #list of cells
print('')

print("total cells in all groups:")
total_ori_sg = sum(ori_sg_prefs)
print(total_ori_sg)
print('')

print("largest group of cells:")
max_ori_sg = np.max(ori_sg_prefs) #max number cells with preferred static orientation
print(max_ori_sg)
print('')

print("percent of total cells:")
per_ori_sg = float(max_ori_sg)/float(total_ori_sg)
print (str(per_ori_sg)+" = roughly "+str(round(per_ori_sg*100))+"%")
print("")

print("most preferred orientation(s):")
for orientation in sorted(VISp_cells_with_numbers["pref_ori_sg"].unique()):
    if(((len(VISp_cells_with_numbers[VISp_cells_with_numbers["pref_ori_sg"]==orientation])))==max_ori_sg):
        print (str(orientation) + " degrees")

cell totals per preferred orientation group:
[494, 551, 568, 580, 600, 690]

total cells in all groups:
3483

largest group of cells:
690

percent of total cells:
0.198105081826 = roughly 20.0%

most preferred orientation(s):
90.0 degrees
